# Conditioned *k*-nearest neighbours

## Description
We propose a probabilistic, token-level, nearest-neighbour algorithm that captures the intuitive notion that the meaning of a word depends on the wider context of the document in which it is found. The word *knight* in a document about history has different connotations than the same word in a document about chess. Consequently, in a nearest-neighbour task, one would expect the word *chivalry*, for example, to be ranked considerably closer to *knight* than *chessboard* in the first instance, and farther in the second.

We use Latent Dirichlet Allocation to represent both documents and tokens as probability distributions over topics (the wider context of documents) and define the conditioned $k$-nn problem as the solution of an *optimal transport* process that allocates vocabulary tokens to query tokens based on their semantic difference interpreted as cost.

## Requirements
Typically, models for recommendation tasks work on the basis of extracting the most representative keywords from an input document, adopting a pre-trained language model $\mathcal{L}$ (GloVe, Word2Vec) to find *semantically similar* search terms, reducing the original task to a nearest-neighbour problem: given a base set $S$ (the model's embedding) and a query $q$ (a token) find the $k$ elements in $S$ that are most similar to $q$.

We observe that, without periodic retraining of the embedding model with new documents, $\mathcal{L}$ remains fixed, consequently, the semantic similarity between the query *knight* from document $X$ and token *chessboard*, is the same whether $X$ pertains to history or chess.

Intuitively, however, for a recommendation task, we would expect to give emphasis to those search terms related to $q$ that more closely belong to the same topic(s) as $X$.

In other words the probability of a search term $w$ being selected as a nearest-neighbour to $q$ should be somewhat correlated to the probability of finding both $q$ and $w$ in documents pertaining to topic z.

## Conditioned $k$-nearest-neighbour problem
Given a set $W=\{w_1, \ldots, w_n\}$, a query $q$ and a distance metric $d$ (typically cosine distance), the nearest-neighbour (nn) problem is solved by ordering the elements in $W$ according to how short their distance is with respect to $q$: the $k$-nn problem simply considers the top $k$ elements.

More generally however, if $Q=\{q_1, \ldots, q_m\}$ is a set of query vectors, the nn problem can be stated in terms of *assignments*, pairing each $w_i \in W$ with a $q_j \in Q$ in such a way as to minimise the total distance.

Let $\Pi$ be an $n$-by-$m$ matrix where a cell $\pi_{ij}=1$ if $w_i$ is assigned to $q_j$ and 0 otherwise, the nn problem becomes:
\begin{equation}
	\min_{\Pi}\sum_{i=1}^n \sum_{j=1}^m \pi_{ij}d(q_j, w_i).
\end{equation}
This is a *pure* assignment where each $w_i$ is assigned to one $q_j$. This can be generalised further with the following intuition.

We may think of $W$ as the inhabitants of a town sprinkled with a few fountains here and there (our set $Q$) with the problem of needing to allocate each town-folk to a fountain by maximising the town-folks' utility whilst complying with any requirements on the fountains.

Town-folks would care to travel the least for a fountain but may have different degrees of demand for water whereas fountains may only be capable of service up to some capacity, beyond which no town-folk can be allocated to it.

Suppose $D$ is the distribution of demand over the town-folks and $C$ is the distribution of capacity over the fountains, the problem of allocating demand to fountains according to capacity and travelling constraints, may be expressed by the following linear program:
\begin{align*}
	\min_{\Pi \in \mathcal{M}(D, C)} &\sum_{i=1}^n \sum_{j=1}^m \pi_{ij}d(q_j, w_i) \\
	&\sum_i \pi_{ij} = C(q_j)\\
	&\sum_j \pi_{ij} = D(w_i)
\end{align*}
where $\mathcal{M}(D,C)$ is the set of all joint distributions between $D$ and $C$ such that $D$ is the first marginal and $C$ the second, $C(q_j)$ is the capacity at $q_j$ and $D(w_i)$ is the demand at $w_i$.

We propose to define the demand $D(w_i)$ at $w_i$ as the conditional probability of token $w_i$ in the vocabulary given document topic $z$ and the capacity $C(q_j)$ at $q_j$ as the normalised conditional probability of query $q_j$ given $z$.

Given a topic $z$, the higher $p(w_i | z)$, the higher the demand, more likely to be distributed across fountains with high capacity - likewise, the higher $p(q_j| z)$ the higher the serviceable capacity.

Intuitively then, words that typically feature heavily in documents about topic $z$ will tend to "attract" each other with higher $\pi_{ij}$ values whilst still being "modulated" by their semantic distance.
>We call any $\Pi$ that minimises the above when $D$ and $C$ are conditional probabilities, a solution to the *conditioned nn* (cnn) problem, and we call selecting the topmost $k$  $\pi_{ij}$ values for each query, a solution to the *conditioned $k$-nn* (cknn) problem.

## Method
### Latent Dirichlet Allocation
We choose an *LDA* model [1] for topic analysis. Given a corpus $\mathcal{C}$ of documents and a number $t$ of latent topics, the *LDA* infers the $n$-by-$t$ matrix $\Phi$, where $n$ is the number of tokens in the vocabulary and cell $\phi_{ij}$ represents the conditional probability $p(w_i| z_j)$ of word $w_i$ given topic $z_j$. The *LDA* also allows us to represent an input document $X$ as a distribution on topics $p(z | X)$ and we consider $z^*= \text{arg}\max p(z|X)$ to be the representative topic of document $X$. The distributions $D$ and $C$ are derived from the *LDA* and the input document such that:

\begin{equation} 
 D(w_i) = \phi_{ix}
\end{equation}
where $x$ is the index of topic $z^*$ and 

\begin{equation}
C(q_j) = \frac{\phi_{jx}}{\sum_{q_i \in Q} \phi_{ix}}
\end{equation}

### Optimal Transport
We note that the *cnn* problem as stated is an *optimal transport* problem [2] and we use an off-the-shelf solver which requires $D$, $C$ and a cost matrix $M$ where cell $m_{ij}=d(q_j, w_i)$ is the *cosine* distance between $w_i$ and $q_j$ in their *GloVe* vector representation. From the resultant solution matrix we select the indices of the top-most $k$ values in each column $j$ to be $k$ nearest-neighbours of query $q_j$ to solve the *cknn* problem.

### TextRank
In order to extract relevant query tokens from an input document $X$ we use an implementation of the *TextRank* algorithm[3]. 

### Conditional probability of a query $q$
Let $C$ be the training corpus for the *LDA* and $X$ an input document with $X \notin C$ then:

\begin{equation}
p(q | z, C \cup X) = p(q | z, C) + p(q | z, X) - p( q | C \cap X)
\end{equation}

If $z = \text{arg}\max p(Z|X)$ then $p(z, X) = 1$ thus we may consider $p(q | z, X) = \frac{N_q}{N_X}$ or simply the number of instances of $q$ divided by the total number of tokens in $X$ whereas $p(q | C \cap X)$ is the number of instances of $q$ divided by the total number of documents containing $q$. Note that if $C$ is large then $p(q| C\cap X)$ becomes relatively negligible therefore we approximate thus:
\begin{equation}
p(q | z, C \cup X) \approx p(q| z, C) + p(q | z, X)
\end{equation}

### Parameters
The *cknn* model requires two parameters, $k$ being the number or required nearest neighbours for each query and $t$, the number of latent topics for the *LDA*.

### Algorithm

> Initialise number of nearest neighbours $k$, number of latent topics $t$, training corpus $\mathcal{C}$, vocabulary $V$ and input document $X$<br>
Compute $\Phi =$ LDA.fit( $t$, $\mathcal{C}$)<br>
Compute $z^* = \text{arg}\max p(z| X) = \text{arg}\max$ LDA.doc_topics( $X$ )<br>
Compute query set $Q = $ textrank.rank( $X$ )<br>
Compute cost matrix $M = $ cosine_distance( GloVe( $V$ ), GloVe( $Q$ ) )<br>
Compute histogram $D = \Phi[:z^*]$<br>
Compute histogram $C = N\cdot\{\Phi[j, z^*] | q_j \in Q\}$ where $N$ is a normalisation factor<br>
Compute the optimal transport $\Pi =$ ot( $D$, $C$, $M$ )<br>
Sort each column of $\Pi$ in descending order<br>
Return the indices of the top $k$ elements from each sorted column of $\Pi$ corresponding to the $k$ nearest neighbours in $V$ for each query.

### Evaluation

#### Topic Relevance Score
Given an input document $X$ of designated topic $z$ and a query $q \in X$, we evaluate the model's output $\text{knn}(q) = \{ w_1, \cdots, w_k\}$ by computing the following *topic relevance score*.

We consider the set $W$ of the 100 topmost relevant words to topic $z$, that is, those words that appear most frequently in documents of topic $z$. 
Intuitively, the contextual model is doing well if it returns neighbours that are relevant to $z$ thus we reward it proportionally to the $z$-relevant neighbours it returns. 

We also consider the 100th percentile of those neighbours that are found in $W$. Recall that the 100th percentile is the score up to and below which all data points fall, thus if the 100th percentile of a set of $k$ nearest neighbours is 62, it means that all the neighbours are located within the top 62 words in $W$.
Let $L = W \cap \text{knn}(q)$, then the topic relevance score is:

\begin{equation}
\tau_q = \frac{|L|}{k} (1 - P_{100}(L))
\end{equation}

The scores from each query from $X$ are then averaged together. The same score is computed for the baseline model, *GloVe* and the two results are compared.

## References
[1] D.M.Blei, A.Y.Ng and M.I.Jordan. 2003. *Latent Dirichlet Allocation*

[2] https://en.wikipedia.org/wiki/Transportation_theory_(mathematics)

[3] R.Mihalcea and P. Tarau. 2004. *TextRank: Bringing Order into Texts* 